## Problem biznesowy  

**Opis zadania**: “Nasz serwis zarabia wtedy, gdy ludzie słuchają muzyki. Jeśli nie wiemy ile czasu będą słuchali jej w przyszłości, to trudno nam rozliczać się z artystami i negocjować z nimi stawki”

### definicja problemu biznesowego
1. Obecna sytuacja: Nie umiemy oszacować czasu słuchania artystów, a tym samym zysków, które przyniosą naszemu serwisowi. Utrudnia to rozliczanie i negocjowanie stawek.
2. Chcemy wprowadzić system szacowania czasu słuchania poszczególnych artystów, który:
    - analizując dane zebrane na temat artystów będzie szacował czas ich słuchania przez użytkowników
    - będzie przedstawiał tę informację w formie czytelnej dla człowkieka, np. w postaci krótkiego tekstu/tabelki

### definicja zadania/zadań modelowania
- Należy zaprojektować model predykcyjny, aproksymujący czas słuchania danego artysty przez użytkowników na podstawie danych o artystach, użytownikach i słuchanych przez nich utworach
- Dane do modelowania:
    - Dane o artystach (gatunek muzyczny)
    - Dane o utworach (częstość i daty przesłuchań, czas trwania, liczba polubień/pominięć)

### definicja wszystkich założeń
- Model będzie realizować predykcje w skali miesiąca (na miesiąc w przód)
- Zadanie będzie realizowane przez model regresji (regresja liniowa/las losowy)
- Model będzie trenowany na dostępnych danych offline

### zaproponowane kryteria sukcesu
- Biznesowe: Różnica w przewidywaniach modelu, a faktycznym czasem słuchania danego artysty nie przekracza 20% w stosunku do rzeczywistej wartości
- Analityczne: Takie samo, jak biznesowe osiągnięte na danych offline


## Uwagi do poprzedniej wersji danych:
1. Ogólne
    - Dużo błędów w danych - przykład: znaki typu \0 w imionach/nazwach, -1 w id artystów
    - Braki w danych - wartości typu null  
    - Bardzo mało danych dotyczących użytkowników w porównaniu do pozostałych
2. artists
    - id
        - Duża liczba niewłaściwych pól = -1 (aż 20%)
        - Pozostałe wartości są unikalne, brak pustych pól
    - name
        - Dużo błędów w nazwach artystów - 11% zawiera kodowanie binarne
        - Nie wszystkie pola są unikalne - około 30 wartości się powtarza maksymalnie 4 razy
    - genres
        - 20% pól jest pustych
        - niektóre gatunki są rozdzielane przecinkami, inne nie - następuje pewna niespójność
        - pytanie czy nie lepiej zastosować kodowanie wskaźnikowe? tzn. utworzyć atrybuty typu _classic, thai, pop_ o kodowaniu binarnym (0/1)
3. sessions:
    - timestamp
        - brak błędów i braków w danych
        - dużo mniej danych z pierwszych 7 miesięcy niż z ostatnich 3
    - user_id
        - 20% pól jest pustych 
        - dość nierównomiernie rozłożone - ponad 2tys danych dla niektórych użytkowników, mniej niż 200 dla innych
    - track_id
        - 20% pól jest pustych
    - event_type
        - brak błędów w danych
        - 20% pól jest pustych
    - session_id
        - brak błędów i braków w danych
4. tracks
    - id
        - 20% pól pustych
        - reszta pól unikalna
    - name
        - 20% pól pustych
        - 19% zawiera kodowanie binarne
    - popularity, duration_ms, explicit i release_date
        - brak pustych pól i błędów
    - id_artist
        - 20% pól pustych 
5. users
    - id
        - zupełnie przypadkowy i niepotrzebny atrybut
    - favourite_genres
        - 14 z 50 pól pustych

## Analiza danych   



**Potrzebne pakiety**

In [2]:
import pandas as pd

### Artists

In [3]:
jsonl_file_path_artists = 'data/IUM23Z_Zad_06_01_v2/artists.jsonl'
df_artists = pd.read_json(jsonl_file_path_artists, lines=True)
print("Ogólny opis danych:\n", "\t liczba pól:", len(df_artists), "\n", df_artists.describe(), "\n")
print("Liczba pustych pól genres: ", df_artists["genres"].isnull().sum(), "\n")


Ogólny opis danych:
 	 liczba pól: 27650 
                             id   name            genres
count                    27650  27650             27650
unique                   27650  27542             13704
top     72578usTM6Cj5qWsi471Nc    TNT  [indonesian pop]
freq                         1      4                74 

Liczba pustych pól genres:  0 



**Uwagi i problemy**:
- Już nie ma pustych pól w atrybucie genres
- Już id przybiera właściwe wartości (nie ma błędów możliwych do wykrycia)
- Id: jest unikalnym atrybutem dyskretnym reprezentującym artystę, powinno wskazać nam, że dane się nie powtarzają -  tzn. kolejne rekordy dotyczą innych osób - pytanie czy to prawda, jako że atrybut "name" się czasem powtarza.   
- Genres: jest atrybutem dyskretnym, reprezentującym gatunki muzyczne. Jest to atrybut "wektorowy". Dane trzeba dostosować - na przykład zakodować wskaźnikowo (dorobić atrybuty binarne typu 'pop', 'rock')
  

    
### Sessions

In [4]:
jsonl_file_path_sessions = 'data/IUM23Z_Zad_06_01_v2/sessions.jsonl'
df_sessions = pd.read_json(jsonl_file_path_sessions, lines=True)
print("Ogólny opis danych:\n", "\t liczba pól:", len(df_sessions), "\n")
print("Ogólny opis atrybutu timestamp:\n", df_sessions["timestamp"].describe(), "\n")
cleaned_df_sessions = df_sessions["user_id"].dropna().astype(str)
print("Liczba wartości user_id: \n", len(cleaned_df_sessions.unique()), "\n")
print("Wartości przybierane przez event_type: \n", df_sessions["event_type"].value_counts(dropna=False), "\n")
print("Liczba pustych pól track_id (null i pustych stringów): ", df_sessions["track_id"].isnull().sum() + (df_sessions["track_id"] == "").sum(), "\n")
print("Liczba rekordów gdzie jednocześnie track_id było puste i event_type nie był równy 'advertisement' lub 'buy_premium': ", df_sessions["track_id"][(df_sessions["track_id"] == "") & (df_sessions["event_type"] != "advertisement") & (df_sessions["event_type"] != "buy_premium")].count(), "\n")
cleaned_df_sessions = df_sessions["track_id"][df_sessions["track_id"] != ""].dropna()
print("Najczęściej przybierane 3 wartości przez track_id: \n", cleaned_df_sessions.value_counts()[:3], "\n")
print("Najrzadziej przybierane 3 wartości przez track_id: \n", cleaned_df_sessions.value_counts()[-3:], "\n")
session_description = df_sessions["session_id"].describe()
print(f"Ogólny opis session_id:\n liczba niepustych pól:{int(session_description['count'])}")


Ogólny opis danych:
 	 liczba pól: 557423 

Ogólny opis atrybutu timestamp:
 count                           557423
mean     2023-08-12 08:44:42.970913280
min         2023-01-16 15:37:42.148516
25%      2023-06-19 14:48:45.506822144
50%      2023-08-22 14:32:56.496897024
75%      2023-10-14 03:02:55.748079104
max         2023-11-27 00:46:08.247335
Name: timestamp, dtype: object 

Liczba wartości user_id: 
 50 

Wartości przybierane przez event_type: 
 event_type
play             306279
skip             126494
like              91647
advertisement     32969
buy_premium          34
Name: count, dtype: int64 

Liczba pustych pól track_id (null i pustych stringów):  33003 

Liczba rekordów gdzie jednocześnie track_id było puste i event_type nie był równy 'advertisement' lub 'buy_premium':  0 

Najczęściej przybierane 3 wartości przez track_id: 
 track_id
7FdUvDkaE24o3FPIWTvzv2    1704
58pgi1RpcU2fVJsxhm7BIr    1675
08mG3Y1vljYA6bvDt4Wqkj    1671
Name: count, dtype: int64 

Najrzadziej przy

**Uwagi i problemy**

- Nadal dużo mniej danych z pierwszych 7 miesięcy niż z ostatnich 3.
- To, track_id jest często puste nie jest problemem. Występuje to tylko w rekordach rejestrujących reklamę lub zakup konta premium - te rekordy nie mają dla nas znaczenia i można je potem usunąć.
- Mamy dane tylko dla 50 użytkowników co będzie problemem, skoro z serwisu korzysta 200 tys.
- "Niezbalansowane" track_id jest dobre - wskazuje, jakie utwory są bardziej popularne 


### Tracks


In [5]:
jsonl_file_path_tracks = 'data/IUM23Z_Zad_06_01_v2/tracks.jsonl'
df_tracks = pd.read_json(jsonl_file_path_tracks, lines=True)
df_tracks["duration_ms"] = df_tracks["duration_ms"].astype('Int64')
df_tracks["popularity"] = df_tracks["popularity"].astype('Int64')
print("Ogólny opis danych:\n", "\t liczba pól:", len(df_tracks), "\n")
print("Opis id: \n", "\tLiczba pól: ", df_tracks["id"].count(), "\n\tLiczba unikalnych wartości: ", len(df_tracks["id"].unique()), "\n")
print("Opis popularity: \n", "\ttyp: dyskretny porządkowy\n\tmax: ", (df_tracks["popularity"].max()), "\n\tmin: ", (df_tracks["popularity"].min()), "\n\tśrednia: ", df_tracks["popularity"].mean(), "\n\tmediana: ", int(df_tracks["popularity"].median()), "\n")
print("Opis duration_ms: \n", "\ttyp: na razie dyskretny\n", df_tracks["duration_ms"].describe().astype('Int64'), "\n")
print("Opis release_date: \n", "\ttyp: dyskretny porządkowy\n\tmax: ", df_tracks["release_date"].max(), "\n\tmin: ", 
      df_tracks["release_date"].min(), "\n\tliczba dat w formacie 'YYYY': ", df_tracks['release_date'].str.count(r'^\d{4}$').sum() ,
      "\n\tliczba dat w formacie 'YYYY-MM': ", df_tracks['release_date'].str.count(r'^\d{4}-\d{2}$').sum(), 
      "\n\tliczba dat w formacie 'YYYY-MM-DD': ", df_tracks['release_date'].str.count(r'^\d{4}-\d{2}-\d{2}$').sum(), "\n")
print("Opis id_artist: \n\ttyp: dyskretny\n", df_tracks["id_artist"].describe(), "\nliczba artist_id występująca tylko raz: ", df_tracks["id_artist"].value_counts()[df_tracks["id_artist"].value_counts() == 1].count(), "\n")

Ogólny opis danych:
 	 liczba pól: 129648 

Opis id: 
 	Liczba pól:  129648 
	Liczba unikalnych wartości:  129648 

Opis popularity: 
 	typ: dyskretny porządkowy
	max:  97 
	min:  0 
	średnia:  29.70743860298655 
	mediana:  29 

Opis duration_ms: 
 	typ: na razie dyskretny
 count     129648
mean      228169
std       109372
min         4000
25%       177400
50%       216787
75%       263267
max      4027622
Name: duration_ms, dtype: Int64 

Opis release_date: 
 	typ: dyskretny porządkowy
	max:  2021-04-16 
	min:  1922 
	liczba dat w formacie 'YYYY':  29162 
	liczba dat w formacie 'YYYY-MM':  621 
	liczba dat w formacie 'YYYY-MM-DD':  99865 

Opis id_artist: 
	typ: dyskretny
 count                     129648
unique                     27650
top       3meJIgRw7YleJrmbpbJK6S
freq                        1185
Name: id_artist, dtype: object 
liczba artist_id występująca tylko raz:  12065 



**Uwagi i problemy**
- atrybut id wskazuje na to, że rekordy "są unikalne" - dotyczą jednej piosenki
- atrybut name nie powinien nam być potrzebny. To, że nie jest unikalny to nie jest problem - wiele piosenek może mieć tę samą nazwę
- atrybut popularity - typ dyskretny porządkowy, będzie na pewno bardzo ważnym atrybutem, wydaje się być poprawny - należy dopytać co oznacza
- atrybut duration_ms - typ dyskretny, będzie na pewno bardzo ważnym atrybutem, wydaje się być poprawny, ze względu na ilość można by zmienić na atrybut ciągły
- atrybut id_artist - typ dyskretny, wskazuje konkrentego artystę (łączy z plikiem "artists.jsonl"), może być problem z tym, że niektórzy artyści mają tylko 1 utwór

### Dokładniejsza analiza
Korelacja liniowa i korelacja Spearmana między popularity danego utworu/liczbą polubień a czasem słuchania z ostatnich x dni (nwm ile tu dni, wszystkie?)

In [41]:
def korelacja_liniowa(a, b):
    n = len(a)
    sum_a = sum(a)
    sum_b = sum(b)
    sum_a_squared = sum([x**2 for x in a])
    sum_b_squared = sum([x**2 for x in b])
    sum_product = sum(x * y for x, y in zip(a, b))
    correlation = (n * sum_product - sum_a * sum_b) /((n * sum_a_squared - sum_a ** 2) * (n * sum_b_squared - sum_b ** 2)) ** 0.5
    return correlation

def korelacja_Spearmana(a, b):
    a_ranks = [sorted(range(len(a)), key=lambda k: a[k]), [0] * len(a)]
    b_ranks = [sorted(range(len(b)), key=lambda k: b[k]), [0] * len(b)]
    for i in range(len(a)):
        a_ranks[1][a_ranks[0][i]] = i + 1
    for i in range(len(b)):
        b_ranks[1][b_ranks[0][i]] = i + 1
    rank_diff = [r1 - r2 for r1, r2 in zip(a_ranks[1], b_ranks[1])]
    n = len(a)
    spearman_correlation1 = 1 - (6 * sum([d ** 2 for d in rank_diff]) / (n * (n ** 2 - 1)))
    return spearman_correlation1

In [45]:
from datetime import timedelta
now = df_sessions["timestamp"].max()
track_full_info= df_tracks.set_index('id')[['popularity', 'duration_ms', 'id_artist']].to_dict(orient='index')

# oblicznaie czasu odtwarzania i liczby polubień dla każdego utworu w ciągu ostatnich 30 dni
for _, value in track_full_info.items():
    value['time_played'] = 0
    value['likes'] = 0
    value['play_without_skip'] = 0
    value['play_with_skip'] = 0
    value['just_play'] = 0
sessions = df_sessions[ (df_sessions['timestamp'] > now - timedelta(days=600)) & (df_sessions['event_type'].isin(['play', 'skip', 'like']))].sort_values(by=['session_id', 'timestamp']).to_dict(orient='records')
index = 0
while index < len(sessions):
    session = sessions[index]
    next_index = index + 1
    if session['event_type'] == 'play':
        track_full_info[session['track_id']]['just_play'] += 1
        if next_index < len(sessions) and sessions[next_index]['session_id'] == session['session_id'] and sessions[next_index]['event_type'] == 'skip':
            track_full_info[session['track_id']]['time_played'] += (sessions[next_index]['timestamp'] - session['timestamp']).total_seconds()*1000
            track_full_info[session['track_id']]['play_with_skip'] += 1
            next_index += 1
        elif next_index < len(sessions) and sessions[next_index]['track_id'] == session['track_id'] and sessions[next_index]['event_type'] == 'like':
            track_full_info[session['track_id']]['likes'] += 1
            next_index += 1
        else:
            track_full_info[session['track_id']]['time_played'] += track_full_info[session['track_id']]['duration_ms'] 
            track_full_info[session['track_id']]['play_without_skip'] += 1
    index = next_index
    
#korelacje

popularity_values = [value['popularity'] for value in track_full_info.values()]
czas_values = [value['time_played'] for value in track_full_info.values()]
likes_values = [value['likes'] for value in track_full_info.values()]   
duration_values = [value['duration_ms'] for value in track_full_info.values()]
play_whithout_skip_values = [value['play_without_skip'] for value in track_full_info.values()]
play_with_skip_values = [value['play_with_skip'] for value in track_full_info.values()]
just_play_values = [value['just_play'] for value in track_full_info.values()]

print(f"Korelacja liniowa między 'popularity' a 'time_played': {korelacja_liniowa(popularity_values, czas_values)}")
print(f"Korelacja liniowa między 'likes' a 'time_played': {korelacja_liniowa(czas_values, likes_values)}")
print(f"Korelacja liniowa między 'duration_ms' a 'time_played': {korelacja_liniowa(czas_values, duration_values)}")
print(f"Korelacja liniowa między 'play_without_skip' a 'time_played': {korelacja_liniowa(czas_values, play_whithout_skip_values)}")
print(f"Korelacja liniowa między 'play_with_skip' a 'time_played': {korelacja_liniowa(czas_values, play_with_skip_values)}")
print(f"Korelacja liniowa między 'just_play' a 'time_played': {korelacja_liniowa(czas_values, just_play_values)}")
print(f"Korelacja Spearmana między 'popularity' a 'time_played': {korelacja_Spearmana(popularity_values, czas_values)}")
print(f"Korelacja Spearmana między 'likes' a 'time_played': {korelacja_Spearmana(czas_values, likes_values)}")
print(f"Korelacja Spearmana między 'duration_ms' a 'time_played': {korelacja_Spearmana(czas_values, duration_values)}")
print(f"Korelacja Spearmana między 'play_without_skip' a 'time_played': {korelacja_Spearmana(czas_values, play_whithout_skip_values)}")
print(f"Korelacja Spearmana między 'play_with_skip' a 'time_played': {korelacja_Spearmana(czas_values, play_with_skip_values)}")
print(f"Korelacja Spearmana między 'just_play' a 'time_played': {korelacja_Spearmana(czas_values, just_play_values)}")


Korelacja liniowa między 'popularity' a 'time_played': 0.10190578696040166
Korelacja liniowa między 'likes' a 'time_played': 0.9662332899881354
Korelacja liniowa między 'duration_ms' a 'time_played': 0.018048553825130452
Korelacja liniowa między 'play_without_skip' a 'time_played': 0.9733269341670152
Korelacja liniowa między 'play_with_skip' a 'time_played': 0.6740716048989647
Korelacja liniowa między 'just_play' a 'time_played': 0.9749719112512834
Korelacja Spearmana między 'popularity' a 'time_played': 0.07095980212453656
Korelacja Spearmana między 'likes' a 'time_played': 0.9986313146707618
Korelacja Spearmana między 'duration_ms' a 'time_played': 0.039799311978347185
Korelacja Spearmana między 'play_without_skip' a 'time_played': 0.991233956778948
Korelacja Spearmana między 'play_with_skip' a 'time_played': 0.9993854274180308
Korelacja Spearmana między 'just_play' a 'time_played': 0.9998102093638994


Możemy stwierdzić że jest mała zależność liniowa między popularity a time_played, bardzo mała między duration_ms a time_played, ale jest bardzo duża między likes a time_played.  
Oznacza to, że mamy dużą zależność liniową między likes atime_played.   
Oddzielnie sprawdziliśmy korelację liniową między likes a popularity - wynosiła ok. 0.07 dla Spearmana i 0.1 dla liniowej.  

In [46]:
artists_full_info = df_artists.set_index('id')[['genres']].to_dict(orient='index')
for _, value in artists_full_info.items():
    value['time_played'] = 0
    value['likes'] = 0
    value['popularity_sum'] = 0
    value['popularity_number'] = 0
    value['play_with_skip'] = 0
    value['play_without_skip'] = 0
    value['just_play'] = 0
for key, value in track_full_info.items():
    artists_full_info[value['id_artist']]['time_played'] += value['time_played']
    artists_full_info[value['id_artist']]['likes'] += value['likes']
    artists_full_info[value['id_artist']]['popularity_sum'] += value['popularity']
    artists_full_info[value['id_artist']]['popularity_number'] += 1
    artists_full_info[value['id_artist']]['play_with_skip'] += value['play_with_skip']
    artists_full_info[value['id_artist']]['play_without_skip'] += value['play_without_skip']
    artists_full_info[value['id_artist']]['just_play'] += value['just_play']
for _, value in artists_full_info.items():
    value['popularity_avg'] = value['popularity_sum'] / value['popularity_number']

#korelacje 
czas_values = [value['time_played'] for value in artists_full_info.values()]
likes_values = [value['likes'] for value in artists_full_info.values()]
popularity_values = [value['popularity_avg'] for value in artists_full_info.values()]
play_with_skip_values = [value['play_with_skip'] for value in artists_full_info.values()]
play_without_skip_values = [value['play_without_skip'] for value in artists_full_info.values()]
just_play_values = [value['just_play'] for value in artists_full_info.values()]
print(f"Korelacja liniowa między 'likes' a 'time_played': {korelacja_liniowa(czas_values, likes_values)}")
print(f"Korelacja liniowa między 'popularity' a 'time_played': {korelacja_liniowa(czas_values, popularity_values)}")
print(f"Korelacja liniowa między 'play_without_skip' a 'time_played': {korelacja_liniowa(czas_values, play_without_skip_values)}")
print(f"Korelacja liniowa między 'play_with_skip' a 'time_played': {korelacja_liniowa(czas_values, play_with_skip_values)}")
print(f"Korelacja liniowa między 'just_play' a 'time_played': {korelacja_liniowa(czas_values, just_play_values)}")
print(f"Korelacja Spearmana między 'likes' a 'time_played': {korelacja_Spearmana(czas_values, likes_values)}")
print(f"Korelacja Spearmana między 'popularity' a 'time_played': {korelacja_Spearmana(czas_values, popularity_values)}")
print(f"Korelacja Spearmana między 'play_without_skip' a 'time_played': {korelacja_Spearmana(czas_values, play_without_skip_values)}")
print(f"Korelacja Spearmana między 'play_with_skip' a 'time_played': {korelacja_Spearmana(czas_values, play_with_skip_values)}")
print(f"Korelacja Spearmana między 'just_play' a 'time_played': {korelacja_Spearmana(czas_values, just_play_values)}")



Korelacja liniowa między 'likes' a 'time_played': 0.9674999136228105
Korelacja liniowa między 'popularity' a 'time_played': 0.08382151130371548
Korelacja liniowa między 'play_without_skip' a 'time_played': 0.926956165016368
Korelacja liniowa między 'play_with_skip' a 'time_played': 0.7524897303233525
Korelacja liniowa między 'just_play' a 'time_played': 0.9807054195626872
Korelacja Spearmana między 'likes' a 'time_played': 0.9999993351036148
Korelacja Spearmana między 'popularity' a 'time_played': 0.15085110034414728
Korelacja Spearmana między 'play_without_skip' a 'time_played': 0.9996562296512925
Korelacja Spearmana między 'play_with_skip' a 'time_played': 0.9999976118571586
Korelacja Spearmana między 'just_play' a 'time_played': 0.9999996904780064


### Dane potrzebne do zadania - konkretne:
- artists   
    - id - pozwala połączyć z innymi tabelami
    - genres - bo niektóre gatunki są częściej słuchane niż inne
- sessions
    - session_id - do obliczania czasu przesłuchania
    - timestamp - wskazuje kiedy coś się działo, dodatkowo ważne jeśli podejdziemy do zadania jako do zadania analizy szeregów czasowych.
    - track_id - pozwala połączyć z innymi tabelami
    - event_type - wskazuje ważne dla nas zdarzenia, w procesie uczenia nie będziemy brali pod uwagę zdarzeń: 'advertisement' i 'buy_premium'
        - like - utwór polubiony, zapewne będzie często jeszcze słuchany
        - play + skip - utwór się jednak nie podoba
        - play bez skip - cały utwór przesłuchany, podobał się 
- tracks
    - id - pozwala połączyć z innymi tabelami
    - popularity - im popularniejszy tym częściej słuchane
    - duration_ms - im dłuższy utwór tym dłuższy czas słuchania
    - release_date - czasem stare utwory nie są już popularne
    

### Dane niepotrzebne do zadania:
- artists
    - name
- track_storage
    - informacje o cache'u, niepotrzebne nam
- sessions
    - session_id
    - user_id
- users
    - nie ma potrzeby wiedzieć kto co słucha tylko co jest słuchane
- tracks
    - name - nic nie wskazuje
    - explicit, key, danceability, ... <- to raczej do przypisywania do konkretnych gatunków, co nie jest naszym zadaniem
